In [ ]:
from owslib import fes
from owslib.csw import CatalogueServiceWeb
from datetime import datetime, timedelta
import pytz

In [ ]:
def _get_csw_connection(endpoint):
    """ Connect to CSW server
    """
    csw = CatalogueServiceWeb(endpoint, timeout=60)
    return csw

def _get_freetxt_search(kw_names):
    """
    Retuns a CSW search object based on input string
    """
    freetxt_filt = fes.PropertyIsLike('apiso:AnyText',  literal=('%s' % kw_names),
                                      escapeChar="\\", singleChar=".",
                                      wildCard="*", matchCase="True")
    return freetxt_filt

def _get_csw_records(csw, filter_list, pagesize=10, maxrecords=1):
    """
    Iterate `maxrecords`/`pagesize` times until the requested value in
    `maxrecords` is reached.
    """
    csw_records = {}
    startposition = 0
    nextrecord = getattr(csw, "results", 1)
    while nextrecord != 0:
        csw.getrecords2(
            constraints=filter_list,
            startposition=startposition,
            maxrecords=pagesize,
            outputschema="http://www.opengis.net/cat/csw/2.0.2",
            esn='full',
        )
        print(csw.results)
        csw_records.update(csw.records)
        if csw.results["nextrecord"] == 0:
            break
        startposition += pagesize + 1  # Last one is included.
        if startposition >= maxrecords:
            break
    csw.records.update(csw_records)

def _fes_date_filter(start, stop, constraint="overlaps"):
    """
    Take datetime-like objects and returns a fes filter for date range
    (begin and end inclusive).
    NOTE: Truncates the minutes!!!
    """
    start = start.strftime("%Y-%m-%d %H:00")
    stop = stop.strftime("%Y-%m-%d %H:00")
    if constraint == "overlaps":
        propertyname = "apiso:TempExtent_begin"
        begin = fes.PropertyIsLessThanOrEqualTo(propertyname=propertyname, literal=stop)
        propertyname = "apiso:TempExtent_end"
        end = fes.PropertyIsGreaterThanOrEqualTo(
            propertyname=propertyname, literal=start
        )
    elif constraint == "within":
        propertyname = "apiso:TempExtent_begin"
        begin = fes.PropertyIsGreaterThanOrEqualTo(
            propertyname=propertyname, literal=start
        )
        propertyname = "apiso:TempExtent_end"
        end = fes.PropertyIsLessThanOrEqualTo(propertyname=propertyname, literal=stop)
    else:
        raise NameError("Unrecognized constraint {}".format(constraint))
    return begin, end

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
import pytz
from ipyleaflet import Map, DrawControl


# Widget definitions
endpoint_widget = widgets.Text(
    value='https://nbs.csw.met.no',
    description='Endpoint:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

bbox_widget = widgets.Text(
    value='-80,60.00,10,90.00',
    description='BBox (comma-separated):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

start_widget = widgets.DatetimePicker(
    description='Start:',
    value=datetime(2025, 3, 5, 0, 0, 0).replace(tzinfo=pytz.utc),
    style={'description_width': 'initial'}
)

end_widget = widgets.DatetimePicker(
    description='End:',
    value=datetime(2025, 6, 15, 0, 0, 0).replace(tzinfo=pytz.utc),
    style={'description_width': 'initial'}
)

kw_names_widget = widgets.Text(
    value='Sentinel-1',
    description='Keyword(s):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Create a map centered roughly on the North Atlantic
m = Map(center=(75, -35), zoom=2, scroll_wheel_zoom=True, layout=widgets.Layout(width='600px', height='400px'))

draw_control = DrawControl(rectangle={"shapeOptions": {"color": "#0000FF"}})
m.add_control(draw_control)

def on_draw(target, action, geo_json):
    if action == 'created' and geo_json['geometry']['type'] == 'Polygon':
        # Extract bbox from polygon coordinates
        coords = geo_json['geometry']['coordinates'][0]
        lons = [c[0] for c in coords]
        lats = [c[1] for c in coords]
        min_lon, max_lon = min(lons), max(lons)
        min_lat, max_lat = min(lats), max(lats)
        # Update the bbox_widget value
        bbox_str = f"{min_lon},{min_lat},{max_lon},{max_lat}"
        bbox_widget.value = bbox_str

draw_control.on_draw(on_draw)


def get_widget_values():
    endpoint = endpoint_widget.value
    bbox = [float(x) for x in bbox_widget.value.split(',')]
    start = start_widget.value
    end = end_widget.value
    kw_names = kw_names_widget.value
    return endpoint, bbox, start, end, kw_names

In [ ]:
ui = widgets.VBox(children=[
    endpoint_widget,
    bbox_widget,
    start_widget,
    end_widget,
    kw_names_widget,
    m
])

In [ ]:
display(ui)

In [ ]:
endpoint, bbox, start, stop, kw_names = get_widget_values()

In [ ]:
endpoint, bbox, start, stop, kw_names

In [ ]:
##### Main start here ##############
#endpoint='https://nbs.csw.met.no'
#bbox = [-80, 60.00, 10, 90.00]
#start = datetime(2025, 3, 5, 00, 00, 00).replace(tzinfo=pytz.utc)
#stop = datetime(2025, 6, 15, 00, 00, 00).replace(tzinfo=pytz.utc)
#kw_names = 'Sentinel-1'
#crs='urn:ogc:def:crs:OGC:1.3:CRS84'



crs='urn:x-ogc:def:crs:EPSG:6.18:4326'

constraints = []
# connect to endpoint
try:
    csw = _get_csw_connection(endpoint)
except Exception as e:
    print("Exception: %s" % str(e))

if kw_names:
    freetxt_filt = _get_freetxt_search(kw_names)
    constraints.append(freetxt_filt)

if all(v is not None for v in [start, stop]):
    begin, end = _fes_date_filter(start, stop)
    constraints.append(begin)
    constraints.append(end)

if bbox:
     bbox_crs = fes.BBox(bbox, crs=crs)
     constraints.append(bbox_crs)

if len(constraints) >= 2:
    filter_list = [fes.And(constraints)]
else:
    filter_list = constraints


_get_csw_records(csw, filter_list, pagesize=10, maxrecords=100)
url_opendap = []

#print(len(csw.records))
for key, value in list(csw.records.items()):
    for ref in value.references:
        #print(ref)
        if ref['scheme'] == 'OPeNDAP:OPeNDAP':
            #if '/2024/' in ref['url']:
            #print(ref['url'])
            url_opendap.append(ref['url'])

In [ ]:
url_opendap